## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC

In [2]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:
inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

,word nº,sentence nº,word,POS tag,entity type
20,20,1.0,from,IN,O
21,21,1.0,that,DT,O
22,22,1.0,country,NN,O
23,24,2.0,family,NNS,O
24,25,2.0,of,IN,O
25,26,2.0,soldier,NNS,O
26,27,2.0,kill,VBN,O
27,28,2.0,in,IN,O
28,29,2.0,the,DT,O
29,30,2.0,conflict,NN,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [4]:
instances = a2.create_instances(inputdata)

In [5]:
instances[20:40]

[Class: gpe Features: ['<start>', '<start>', '<start>', '<start>', '<start>', 'this', 'week', 'restart', 'part', 'of'],
 Class: geo Features: ['of', 'the', 'conversion', 'process', 'at', 'nuclear', 'plant', '<end>', '<end>', '<end>'],
 Class: gpe Features: ['<start>', '<start>', '<start>', '<start>', '<start>', 'official', 'say', 'they', 'expect', 'to'],
 Class: tim Features: ['sealed', 'sensitive', 'part', 'of', 'the', 'after', 'an', 'iaea', 'surveillance', 'system'],
 Class: org Features: ['of', 'the', 'plant', 'wednesday', 'after', 'surveillance', 'system', 'begin', 'function', '<end>'],
 Class: org Features: ['<start>', '<start>', '<start>', '<start>', '<start>', 'union', 'with', 'u.s.', 'backing', 'have'],
 Class: gpe Features: ['<start>', '<start>', 'the', 'european', 'union', 'backing', 'have', 'threaten', 'to', 'refer'],
 Class: gpe Features: ['u.s.', 'backing', 'have', 'threaten', 'to', 'to', 'the', 'u.n.', 'security', 'council'],
 Class: org Features: ['threaten', 'to', 'refe

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [6]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,attempted,thousand,of,demonstrator,have,march,to,protest,the,war,...,relocate,jowhar,pilgrimage,jean,bertrand,aristide,yvon,neptune,privert,downgrade
20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[6353.,    0.,    0., ...,    0.,    0.,    0.],
        [2448.,    0.,    0., ...,    0.,    0.,    0.],
        [6348.,    0.,    0., ...,    0.,    0.,    0.],
        ...,
        [6312.,    0.,    0., ...,    0.,    0.,    0.],
        [4247.,    0.,    0., ...,    0.,    0.,    0.],
        [6229.,    0.,    0., ...,    0.,    0.,    0.]]),
 0       tim
 1       tim
 2       geo
 3       geo
 4       tim
        ... 
 5530    gpe
 5531    geo
 5532    geo
 5533    gpe
 5534    org
 Name: class, Length: 5535, dtype: object,
 array([[4.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [9.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [1.400e+01, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        ...,
        [6.905e+03, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e+00],
        [6.910e+03, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
         0.000e

In [8]:
len(test_y) / (len(test_y) + len(train_y))

0.20002890591125885

In [9]:
len(test_X) / (len(test_X) + len(train_X))

0.20002890591125885

In [10]:
test_y[0]

'geo'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [11]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

/usr/local/lib64/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
train_predictions

array(['nat', 'nat', 'nat', ..., 'nat', 'nat', 'nat'], dtype=object)

In [13]:
train_y

0       tim
1       tim
2       geo
3       geo
4       tim
       ... 
5530    gpe
5531    geo
5532    geo
5533    gpe
5534    org
Name: class, Length: 5535, dtype: object

In [14]:
test_predictions

array(['tim', 'geo', 'geo', ..., 'nat', 'nat', 'nat'], dtype=object)

In [15]:
test_y

0       geo
1       gpe
2       geo
3       geo
4       gpe
       ... 
1379    per
1380    per
1381    geo
1382    tim
1383    gpe
Name: class, Length: 1384, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [16]:
a2.confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,0,0,4,2,5,1,0,0
eve,0,0,3,1,7,0,0,0
geo,0,0,94,11,319,2,1,2
gpe,0,0,40,32,178,11,1,1
nat,0,0,1,1,1,0,0,0
org,0,0,30,13,180,9,0,1
per,0,0,27,11,155,12,0,0
tim,0,0,44,4,174,4,1,1


In [17]:
a2.confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,1,0,15,3,20,2,0,0
eve,0,0,9,2,21,2,0,0
geo,0,0,383,32,1217,7,0,2
gpe,0,0,154,127,667,16,1,2
nat,0,0,5,1,10,1,0,0
org,0,0,115,40,780,61,2,4
per,0,0,88,34,750,16,13,1
tim,0,0,180,14,698,18,1,20


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')